In [16]:
from brt.routers.inference import make_homo_proto_tensor_cls

make_homo_proto_tensor_cls()

from brt.routers.app import RandHomoFusedScatterRouter
from brt.routers.inference import HomoFusedGatherRouter
import brt.nn as nn
import torch


class FusedMoE(nn.Module):
    def __init__(self, expert_num):
        super().__init__()
        self.scatter_router = RandHomoFusedScatterRouter(
            dst_num=expert_num,
            supported_capacities=[
                2,
                4,
                8,
                16,
                32,
                64,
                128,
                256,
                512,
                1024,
                2048,
                4096,
                8192,
            ],
        )
        self.gather_router = HomoFusedGatherRouter(dst_num=expert_num)

    def forward(self, inputs):
        route_results = self.scatter_router(inputs)
        # print(route_results)
        route_results = self.gather_router(route_results)
        return route_results


fused_moe = FusedMoE(expert_num=8).cuda()

input_tensor = torch.rand((1024, 64)).cuda()
# print(input_tensor)
output_tensor = fused_moe(input_tensor)
# print(output_tensor)
print(torch.allclose(output_tensor, input_tensor))


generate_local_indices elapsed time: 0.088
route_with_local_indices elapsed time: 0.199
route_back_with_local_indices elapsed time: 0.080
False
